In [1]:
from decouple import config
from qiskit import IBMQ
from datetime import datetime
import pprint
import json

In [2]:
IBMQ.load_account()
IBMQ.providers()
provider = IBMQ.get_provider(hub='strangeworks-hub', group='qc-com', project='runtime')
print(provider.backends())

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='strangeworks-hub', group='qc-com', project='runtime')>, <IBMQBackend('ibm_nairobi') from IBMQ(hub='strangeworks-hub', group='qc-com', project='runtime')>]


In [3]:
import pandas as pd

df = pd.read_csv('qiskit_runtime/qka/aux_file/dataset_graph7.csv',sep=',', header=None) # alterative problem: dataset_graph10.csv
data = df.values

In [4]:
import numpy as np

# choose number of training and test samples per class:
num_train = 10
num_test = 10

# extract training and test sets and sort them by class label
train = data[:2*num_train, :]
test = data[2*num_train:2*(num_train+num_test), :]

ind=np.argsort(train[:,-1])
x_train = train[ind][:,:-1]
y_train = train[ind][:,-1]

ind=np.argsort(test[:,-1])
x_test = test[ind][:,:-1]
y_test = test[ind][:,-1]

In [5]:
from qiskit_runtime.qka import FeatureMap

d = np.shape(data)[1]-1                                         # feature dimension is twice the qubit number

em = [[0,2],[3,4],[2,5],[1,4],[2,3],[4,6]]                      # we'll match this to the 7-qubit graph
# em = [[0,1],[2,3],[4,5],[6,7],[8,9],[1,2],[3,4],[5,6],[7,8]]  # we'll match this to the 10-qubit graph

fm = FeatureMap(feature_dimension=d, entangler_map=em)          # define the feature map
initial_point = [0.1]                                           # set the initial parameter for the feature map

In [6]:
from qiskit.tools.visualization import circuit_drawer
circuit_drawer(fm.construct_circuit(x=x_train[0], parameters=initial_point),
               output='text', fold=200)

┌──────────┐   ┌──────────────┐ ┌────────────┐                                                 
q_0: ┤ RY(-0.1) ├─■─┤ RZ(-0.38383) ├─┤ RX(3.3378) ├─────────────────────────────────────────────────
     ├──────────┤ │ └──────────────┘ └────────────┘┌──────────────┐ ┌──────────────┐                
q_1: ┤ RY(-0.1) ├─┼────────────────────────■───────┤ RZ(-0.11811) ├─┤ RX(-0.20449) ├────────────────
     ├──────────┤ │                        │       └──────────────┘ └┬────────────┬┘┌─────────────┐ 
q_2: ┤ RY(-0.1) ├─■────────■───────────────┼──────────────■──────────┤ RZ(3.4802) ├─┤ RX(0.15495) ├─
     ├──────────┤          │               │              │         ┌┴────────────┤ ├─────────────┴┐
q_3: ┤ RY(-0.1) ├─■────────┼───────────────┼──────────────■─────────┤ RZ(0.34764) ├─┤ RX(-0.54085) ├
     ├──────────┤ │        │               │                        ├─────────────┤ ├──────────────┤
q_4: ┤ RY(-0.1) ├─■────────┼───────────────■──────────────■─────────┤ RZ(0.34312) ├─┤ RX(-0.14015) ├
     ├──────────┤          │        ┌─────────────┐       │        ┌┴─────────────┴┐└──────────────┘
q_5: ┤ RY(-0.1) ├──────────■────────┤ RZ(0.51497) ├───────┼────────┤ RX(-0.029293) ├────────────────
     ├──────────┤                   └─────────────┘       │        └┬──────────────┤┌─────────────┐ 
q_6: ┤ RY(-0.1) ├─────────────────────────────────────────■─────────┤ RZ(-0.42725) ├┤ RX(0.44115) ├─
     └──────────┘                                                   └──────────────┘└─────────────┘

In [7]:
C = 1                                                           # SVM soft-margin penalty
maxiters = 10                                                   # number of SPSA iterations



In [13]:
initial_layout = [0, 1, 2, 3, 4, 5, 6]                    # see figure above for the 7-qubit graph
# initial_layout = [9, 8, 11, 14, 16, 19, 22, 25, 24, 23]       # see figure above for the 10-qubit graph

In [14]:
print(provider.runtime.program('quantum-kernel-alignment'))



quantum-kernel-alignment:
  Name: quantum-kernel-alignment
  Description: Quantum kernel alignment algorithm that learns, on a given dataset, a quantum kernel maximizing the SVM classification margin.
  Version: 1
  Creation date: 2021-05-06T14:56:53.000000
  Max execution time: 28800
  Input parameters:
    - feature_map:
      Description: An instance of FeatureMap in dictionary format used to map classical data into a quantum state space.
      Type: dict
      Required: True
    - data:
      Description: NxD array of training data, where N is the number of samples and D is the feature dimension.
      Type: numpy.ndarray
      Required: True
    - labels:
      Description: Nx1 array of +/-1 labels of the N training samples.
      Type: numpy.ndarray
      Required: True
    - initial_kernel_parameters:
      Description: Initial parameters of the quantum kernel. If not specified, an array of randomly generated numbers is used.
      Type: numpy.ndarray
      Required: False
    -

In [15]:
def interim_result_callback(job_id, interim_result):
    print(f"interim result: {interim_result}\n")

In [16]:
# backend = provider.get_backend('ibmq_qasm_simulator')
backend = provider.get_backend('ibm_nairobi')


In [21]:
program_inputs = {
    'feature_map': fm,
    'data': x_train,
    'labels': y_train,
    'initial_kernel_parameters': initial_point,
    'maxiters': maxiters,
    'C': C,
    'initial_layout': initial_layout
}

options = {'backend_name': backend.name()}

job = provider.runtime.run(program_id="quantum-kernel-alignment",
                              options=options,
                              inputs=program_inputs,
                              callback=interim_result_callback,
                              )

print(job.job_id())
result2 = job.result()

c3lhkm1kjfmd50ohql90


runtime_job._start_websocket_client:WARNING:2021-07-11 11:35:26,424: An error occurred while streaming results from the server for job c3lhkm1kjfmd50ohql90:
Traceback (most recent call last):
  File "/Users/dylan/repos/qiskit-qka-runtime-example/venv/lib/python3.8/site-packages/qiskit/providers/ibmq/runtime/runtime_job.py", line 289, in _start_websocket_client
    self._ws_client.job_results()
  File "/Users/dylan/repos/qiskit-qka-runtime-example/venv/lib/python3.8/site-packages/qiskit/providers/ibmq/api/clients/runtime_ws.py", line 74, in job_results
    self.stream(url=url, retries=max_retries, backoff_factor=backoff_factor)
  File "/Users/dylan/repos/qiskit-qka-runtime-example/venv/lib/python3.8/site-packages/qiskit/providers/ibmq/api/clients/base.py", line 211, in stream
    raise WebsocketError(error_message)
qiskit.providers.ibmq.api.exceptions.WebsocketError: 'Max retries exceeded: Failed to establish a websocket connection. Error: Traceback (most recent call last):\n  File "/Us

In [23]:
program_inputs = {
    'feature_map': fm,
    'data': x_train,
    'labels': y_train,
    'initial_kernel_parameters': initial_point,
    'maxiters': maxiters,
    'C': C,
    'initial_layout': initial_layout
}

options = {'backend_name': 'ibmq_qasm_simulator'}

job = provider.runtime.run(program_id="quantum-kernel-alignment",
                              options=options,
                              inputs=program_inputs,
                              callback=None,
                              )

print(job.job_id())
result3 = job.result()

c3lm8gbma37p8k0h4hh0


In [20]:
result

{'aligned_kernel_parameters': array([0.51280987]),
 'aligned_kernel_matrix': array([[1.00000000e+00, 4.24804688e-02, 5.24902344e-02, 1.23535156e-01,
         1.96044922e-01, 1.29394531e-02, 1.31835938e-02, 3.07617188e-02,
         4.56542969e-02, 1.12670898e-01, 2.53906250e-02, 7.12890625e-02,
         1.06811523e-01, 6.53076172e-02, 2.16064453e-02, 4.94384766e-02,
         4.56909180e-01, 2.56347656e-02, 2.77099609e-01, 1.09863281e-01],
        [4.24804688e-02, 1.00000000e+00, 3.78417969e-03, 1.18408203e-02,
         2.63671875e-02, 1.39160156e-02, 2.13134766e-01, 4.88281250e-02,
         5.79833984e-02, 1.38793945e-01, 5.88378906e-02, 2.22167969e-02,
         1.57470703e-02, 1.77734375e-01, 7.08007812e-02, 2.18505859e-02,
         3.58886719e-02, 9.89990234e-02, 7.34863281e-02, 1.26586914e-01],
        [5.24902344e-02, 3.78417969e-03, 1.00000000e+00, 1.86279297e-01,
         1.43920898e-01, 1.16821289e-01, 1.03759766e-02, 5.22460938e-02,
         2.46582031e-02, 1.24511719e-02, 2.319